# Analise de Latência das aplicações na Mali

In [1]:
import pandas as pd

import glob

path='c:\\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Latência'

files = [f for f in glob.glob(path + "**/*.txt", recursive=True)]

for f in files:
    print(f)

c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Latência\2DCONV_GPU_177000000_IdlesFreq_1GHZ_log_tempo.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Latência\2DCONV_GPU_266000000_IdlesFreq_1GHZ_log_tempo.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Latência\2DCONV_GPU_350000000_IdlesFreq_1GHZ_log_tempo.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Latência\2DCONV_GPU_420000000_IdlesFreq_1GHZ_log_tempo.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Latência\2DCONV_GPU_480000000_IdlesFreq_1GHZ_log_tempo.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Latência\2DCONV_GPU_543000000_IdlesFreq_1GHZ_log_tempo.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Latência\2DCONV_GPU_600000000_IdlesFreq_1GHZ_log_tempo.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Latência\2MM_GPU_177000000_IdlesFreq_1GHZ_log_tempo.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de 

In [2]:
def group_results(file, name):
    df_temp = pd.DataFrame()
    dict_temp = {}    
    
    #df_mean = df.groupby('Name', axis=0).mean().reset_index()
    for line in file: 
        if ":" in line:
            key, value = line.split(':')[0], line.split(':')[1]
            dict_temp[key] =  float(value)
            if "Free mem" in key:
                #last variable can add to the dataframe
                #print(dict_temp)
                df_temp = df_temp.append(dict_temp, ignore_index=True)
                dict_temp = dict.fromkeys(dict_temp, 0)
            
    df_temp = df_temp.mean().to_dict()
    
    app = name.split('\\')[-1].split('_')[0]
    dev = name.split('_')[1]
    dev_freq = int(name.split('_')[2])
    
    df_temp['app'] = app
    df_temp['dev freq'] = dev_freq
    df_temp['dev'] = str(dev)
    
    return df_temp

In [3]:
df = pd.DataFrame()

for path in files:
    log_data=open(path,'r')
    temp_dict = group_results(log_data, path)
    df = df.append(temp_dict, ignore_index=True)

In [4]:
df.columns

Index(['CPU Kernel \t runtime', 'Clean  up    runtime', 'Data Alloc   runtime',
       'Free memory  runtime', 'GPU Kernel \t runtime',
       'Init Array \t runtime', 'Kernels \t runtime', 'Kernels Init runtime',
       'Launch Kernl runtime', 'Memory Init  runtime',
       'Non-Matching CPU-GPU Outputs Beyond Error Threshold of 1.05 Percent',
       'Prog Load    runtime', 'Read Kernel  runtime', 'Sequential \t runtime',
       'app', 'dev', 'dev freq',
       'Non-Matching CPU-GPU Outputs Beyond Error Threshold of 0.05 Percent'],
      dtype='object')

In [5]:
to_remove = ['CPU Kernel \t runtime','Kernels \t runtime','GPU Kernel \t runtime',
             'dev','Non-Matching CPU-GPU Outputs Beyond Error Threshold of 0.05 Percent',
             'Non-Matching CPU-GPU Outputs Beyond Error Threshold of 1.05 Percent',
             'Sequential \t runtime']

df_temp = df.loc[:,~df.columns.isin(to_remove)]
df_temp.set_index(['dev freq'], inplace=True)
df_temp['total'] = df_temp.sum(axis=1)
df_temp = df_temp.reset_index()

df_temp.columns = ['Freq', 'Clean_up', 'Data_alloc', 'Free_mem', 'Init_arr', 'Kernels_init', 'Launch_kernel', 'Memory_init', 'Prog_load', 'Read_kernel','App', 'Total']
df_temp


c:\users\dougw\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Freq,Clean_up,Data_alloc,Free_mem,Init_arr,Kernels_init,Launch_kernel,Memory_init,Prog_load,Read_kernel,App,Total
0,177000000.0,0.000486,0.000168,0.042051,4.672837,0.068646,0.278984,0.389990,0.078682,0.000163,2DCONV,5.532006
1,266000000.0,0.000479,0.000170,0.041543,4.666530,0.068287,0.187486,0.389180,0.078402,0.000163,2DCONV,5.432240
2,350000000.0,0.000493,0.000166,0.042361,4.669196,0.068678,0.144280,0.395281,0.078124,0.000166,2DCONV,5.398746
3,420000000.0,0.000478,0.000170,0.043359,4.666857,0.068794,0.121581,0.393787,0.078289,0.000169,2DCONV,5.373485
4,480000000.0,0.000482,0.000165,0.042823,4.669286,0.069037,0.107485,0.396796,0.078488,0.000158,2DCONV,5.364720
5,543000000.0,0.000477,0.000167,0.042653,4.669331,0.068340,0.096196,0.394513,0.078165,0.000162,2DCONV,5.350006
6,600000000.0,0.000485,0.000170,0.042220,4.669334,0.068314,0.088114,0.391474,0.078186,0.000166,2DCONV,5.338462
7,177000000.0,0.000583,0.000230,0.027245,0.261158,0.069010,102.484295,0.352243,0.101634,0.000152,2MM,103.296550
8,266000000.0,0.000585,0.000220,0.026879,0.263381,0.068999,68.909871,0.355684,0.101555,0.000154,2MM,69.727329
9,350000000.0,0.000583,0.000220,0.027784,0.262150,0.068715,52.955098,0.355333,0.101412,0.000152,2MM,53.771446


In [6]:
df_temp.columns

Index(['Freq', 'Clean_up', 'Data_alloc', 'Free_mem', 'Init_arr',
       'Kernels_init', 'Launch_kernel', 'Memory_init', 'Prog_load',
       'Read_kernel', 'App', 'Total'],
      dtype='object')

In [7]:
#%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from plotnine import *

apps = list(set(df_temp['App']))
dict_plot = {}

for app in apps:
    df_plot = df_temp[df_temp['App'] == app]
    df_plot = df_plot.loc[:, df_plot.columns.difference(['Freq'])] 
    #dict_plot[app] = ggplot(df_plot,aes(x='Freq',y='Total'))+geom_line()
    df_plot.plot()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel("Frequency (x 100MHZ)")
    plt.ylabel("Seconds")

#ggplot(df_plot[df_temp['App'] == '2MM'],aes(x='Freq',y='Total'))
#+ geom_point()

#for i,v in zip(dict_plot.keys(),dict_plot.values()):
#    print(i)
#    print(v)

# Analise Energia

In [8]:
path='c:\\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Energia'

files = [f for f in glob.glob(path + "**/*.txt", recursive=True)]

for f in files:
    print(f)

c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Energia\2DCONV_GPU_177000000_IdlesFreq_1GHZ_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Energia\2DCONV_GPU_266000000_IdlesFreq_1GHZ_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Energia\2DCONV_GPU_350000000_IdlesFreq_1GHZ_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Energia\2DCONV_GPU_420000000_IdlesFreq_1GHZ_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Energia\2DCONV_GPU_480000000_IdlesFreq_1GHZ_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Energia\2DCONV_GPU_543000000_IdlesFreq_1GHZ_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Energia\2DCONV_GPU_600000000_IdlesFreq_1GHZ_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\Logs de Energia\2MM_GPU_177000000_IdlesFreq_1GHZ_log_energia.txt
c:\Users\dougw\Desktop\Analise-Polybench-Mali-T628\

In [9]:
apps

['CORR',
 'COVAR',
 'SYRK',
 'SYR2K',
 'GESUMMV',
 'FDTD-2D',
 '2DCONV',
 '3DCONV',
 'BICG',
 'GEMM',
 'MVT',
 '3MM',
 'ATAX',
 '2MM',
 'GRAMSCHM']